In [1]:
import time
import datetime
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
#Set start time for tracking
start_time = time.time()

#URL, numbers appended corresponding to CHEMBLid to go to molecule report card
CHEMBL = "https://www.ebi.ac.uk/chembl/beta/compound_report_card/CHEMBL"

#Chrome set as driver
driver = webdriver.Chrome()

#List to store SMILES strings into
SMILESdata = []

#Counter to show number of SMILES strings actually saved: not all CHEMBL ids correspond to a molecule
counter = 0
#Loop through all CHEMBL ids
for i in range(20000):
    #Catch error if index error (resulting from server error on website) occurs, continue running
    while True: 
        try:
            #Append number to end of URL for webpage of the CHEMBLid
            CHEMBL+=str(i+1)
    
            #Get URL
            driver.get(CHEMBL)
    
            #Allow webpage to load
            time.sleep(.25)
    
            #Encode webpage
            content = driver.page_source.encode('utf-8').strip()
    
            #Parse content 
            CHEMBLsoup = BeautifulSoup(content, "html.parser")
    
            #Find table, row, and column of SMILES string
            table = CHEMBLsoup.find_all("table", class_="compact-table hide-on-small-only")[0]
            table_body = table.find("tbody")
            row = table_body.find_all('tr')[1]
            col = row.find("td", class_="BCK-CanonicalSmiles")
    
            #Get SMILES string: cutting off characters faster than performing another search for SMILES string in webpage
            SMILES = str(col)[140:].split(">",1)[0]
            length = len(str(col)[140:].split(">",1)[0])
            SMILES = SMILES[:length-1]
    
            #Assert a SMILES string exists, append to SMILESdata if it does, and update counter to show molecules saved
            if ((len(SMILES))!=0):
                SMILESdata.append(SMILES)
                counter+=1
        
            #Display time elapsed from start, SMILES saved, and run number
            elapsed = time.time() - start_time
            hours = int(elapsed/3600)
            minutes = int(int(elapsed/60)%60)
            seconds = int(elapsed%60)
            print("Run number: ", i, " | SMILES saved:", counter, " | Time elapsed:", hours, "hours", minutes, "minutes", seconds, "seconds")
    
            #Reset URL to one without CHEMBL id to allow for loop
            CHEMBL = "https://www.ebi.ac.uk/chembl/beta/compound_report_card/CHEMBL"
            
            #Save SMILES strings collected to file every 1000 runs
            if ((i % 1000) == 0):
                #Save SMILES strings to text file
                data = open("SMILESdata.txt", "a")
                for i in SMILESdata:
                    data.write("%s\n" % i)
                    data.close()
            break
        except IndexError:
            #Index errors resulted from server errors, catch exception and continue running
            print("Server error occurred.")
            
            #Display time elapsed from start, SMILES saved, and run number
            elapsed = time.time() - start_time
            hours = int(elapsed/3600)
            minutes = int(int(elapsed/60)%60)
            seconds = int(elapsed%60)
            print("Run number: ", i, " | SMILES saved:", counter, " | Time elapsed:", hours, "hours", minutes, "minutes", seconds, "seconds")
            
            #Reset URL to one without CHEMBL id to allow for loop
            CHEMBL = "https://www.ebi.ac.uk/chembl/beta/compound_report_card/CHEMBL"
            
#Close browser
driver.close()

#Save SMILES strings to text file
data = open("SMILESdata.txt", "a")
for i in SMILESdata:
    data.write("%s\n" % i)
data.close()

Run number:  0  | SMILES saved: 1  | Time elapsed: 0 hours 0 minutes 10 seconds
Run number:  1  | SMILES saved: 2  | Time elapsed: 0 hours 0 minutes 12 seconds
Run number:  2  | SMILES saved: 3  | Time elapsed: 0 hours 0 minutes 14 seconds
Run number:  3  | SMILES saved: 4  | Time elapsed: 0 hours 0 minutes 18 seconds
